# Bandits in the Real World: Complexity and Complications

We think that bandit algorithms are a very powerful set of tools for building better websites. But they are not a panacea. Bandit algorithms can break if the world you’re exploring is more complicated than the world we’ve used in testing algorithms. Let’s list some questions you should ask yourself when deploying bandit algorithms in the wild:
- How sure are you that you won’t subtly corrupt your deployment code?
- How many different tests are you planning to run simultaneously? Will these tests interfere with each other? Will starting a new test while another one is already running corrupt its results?
- How long do you plan to run your tests?
- How many users are you willing to expose to non-preferred versions of your site?
- How well-chosen is your metric of success?
- How are the arms you’re measuring related to one another?
- What additional information about context do you have when choosing arms? Do you have demographics based on browser information? Does your site have access to external information about people’s tastes in products you might advertise to them?
- How much traffic does your site receive? Is the system you’re building going to scale up? How much traffic can your algorithm handle before it starts to slow your site down?
- How much will you have to distort the setup we’ve introduced when you admit that visitors to real websites are concurrent and aren’t arriving sequentially as in our simulations?

## A/A Testing

Banal though it sounds, the real world is a very complicated place. The idealised scenarios we’ve been using in our Monte Carlo tests for evaluating the performance of different bandit algorithms are much simpler than real deployment scenarios. In the real world, both the observations you collect and the code you write to collect those observations are likely to be more complex than you realise.

The result of this complexity is that you may observe differences between two arms that are totally illusory at the same time that the data you collect will insist that those differences are significant. Researchers at Microsoft published a paper on “Trustworthy Online Controlled Experiments: Five Puzzling Outcomes Explained” that does a good job of describing some problems you are likely to run into if you deploy a bandit algorithm on your site.

One of their solutions to the problems that will come up is particularly counter-intuitive, but worth considering before we address other concerns. They want you to run A/A testing. In other words, you should use the code you develop for assigning users to arms, but then define two different arms that are actually both the same user experience. If you find differences between these two identical arms, you need to substantially temper your claims about differences between other arms. This illusory difference between two A arms may indicate that there’s a bug in your code or a mistake in the way you’re analyzing your data. But it may also indicate that your test is running in a context that is subtly different from the assumptions we’ve been implicitly making when setting up our algorithms and simulations.

Even if you try A/A testing and don’t find any worring issues, this approach provides a useful way to estimate the actual variability in your data before trying to decide whether the differences found by a bandit algorithm are real. And that matters a lot if your plan is to use a bandit algorithm not as a permanent feature of your site, but as a one-off experiment.

## Running Concurrent Experiments

While we’ve discussed algorithms that can cope with a few arms that are well-separated, many real-world websites will end up running many different experiments simultaneously. These experiments will end up overlapping: a site may use A/B testing to compare two different logo colors while also using A/B testing to compare two different fonts. Even the existence of one extra test that’s not relating to the arms you’re comparing can add a lot of uncertainty into your results. Things may still work out well. But your experiments may also turn out very badly if the concurrent changes you’re making to your site don’t play well together and have strange interactions.

In an ideal world, concurrency issues raised by running multiple experiments at once won’t come up. You’ll be aware that you have lots of different questions and so you would plan all of your tests in one giant group. Then you would define your arms in terms of the combinations of all the factors you want to test: if you were testing both colors and fonts, you’d have one arm for every color/font pair.

This ideal world fails not only because people get sparks of inspiration that make them change course over time. It also fails because the number of arms you would need to test can quickly blow up if you start combining the different factors you want to test into separate pairs. Of course, if you don’t keep track of other tests, you may end up with a large number of puzzling results that are all artifacts of running so many experiments simultaneously.

The best solution to this is simple: _try your best to keep track of all of the experiments each user is a part of and include this information in your analyses of any single experiment_

## Continuous Experimentation vs. Periodic Testing

Are you planning to run tests for a while to decide which approaches are best? Are you then going to stop running new experiments after you’ve made that decision? In that case, A/B testing may often be wise if you have a similar set of proposed changes that would become arms in your Multiarmed Bandit setup. If you’re doing short-term experiments, it’s often not so important to avoid testing inferior strategies because the consequences aren’t so bad.

But if you’re willing to let your experiments run much longer, turning things over to a bandit algorithm can be a huge gain because the algorithm will automatically start to filter out inferior designs over time without requiring you to make a judgment call. Whether this is a good thing or not really depends on the details of your situation. But the general point stands: _bandit algorithms look much better than A/B testing when you are willing to let them run for a very long time_. If you’re willing to have your site perpetually be in a state of experimentation, bandit algorithms will be many times better than A/B testing.

A related issue to the contrast between continuous experimentation versus short periods of experimentation is the question of how many users should be in your experiments. You’ll get the most data if you put more users into your test group, but you risk alienating more of them if you test something that’s really unpopular. The answers to this question don’t depend on whether you’re using bandit algorithm or A/B testing, but the answers will affect how well a bandit algorithm can work in your setting. If you run a bandit algorithm on a very small number of users, you may end up with too little data about the arms that the algorithm decided were inferior to make very strong conclusions about them in the future. A/B testing’s preference for balancing people across arms can be advantageous if you aren’t going to gather a lot of data.

## Bad Metrics of Success

The core premise of using a bandit algorithm is that you have a well-defined measure of reward that you want to maximize. A real business is much more complicated than this simple setup might suggest. One potentially fatal source of increased complexity is that optimizing short-term click-through rates may destroy the long-term retainability of your users. Greg Linden, one of the earlier developers of A/B testing tools at Amazon, says that this kind of thing actually happened to Amazon in the 1990’s when they first started doing automatic A/B testing. The tools that were ostensibly optimizing their chosen metric were actually harming Amazon’s long-term business. Amazon was able to resolve the situation, but the problem of optimizing the wrong metric of success is so ubiquitous that it’s likely other businesses have lost a great deal more than Amazon did because of poorly chosen metrics.

Unfortunately, there’s no algorithmic solution to this problem. Once you decide to start working with automated metrics, you need to supplement those systems by exercising human judgment and making sure that you keep an eye on what happens as the system makes changes to your site.

Monitoring many different metrics you think are important to your business is probably the best thing you can hope do. For example, creating an aggregate site well-being score that simply averages together a lot of different metrics you want to optimise may often be a better measure of success than any single metric you would try in isolation.

## Scaling Problems with Good Metrics of Success

Even if you have a good metric of success, like the total amount of purchases made by a client over a period of a year, the algorithms described in this book may not work well unless you rescale those metrics into the 0-1 space we’ve used in our examples. The reasons for this are quite boring: some of the algorithms are numerically unstable, es‐ pecially the softmax algorithm, which will break down if you start trying to calculate things like `exp(10000.0)`. You need to make sure that you’ve scaled the rewards in your problem into a range in which the algorithms will be numerically stable. If you can, try to use the 0-1 scale we’ve used, which is, as we briefly noted earlier, an absolute requirement if you plan on using the UCB1 algorithm.

## Intelligent Initialization of Values

In the section on the epsilon-Greedy algorithm, we mentioned how important it is to consider how you initialize the values of arms you’ve never explored. In the real world, you can often do this using information you have before ever deploying a bandit algorithm. This smart initialisation can happen in two ways.

First, you can use the historical metrics for the control arm in your bandit algorithm. Whatever arm corresponds to how your site traditionally behaved can be given an initial value based on data from before you let the bandit algorithm loose. In addition, you can initialize all of the unfamiliar arms using this same approach.

Second, you can use the amount of historical data you have to calibrate how much the algorithm thinks you know about the historical options. For an algorithm like UCB1, that will strongly encourage the algorithm to explore new options until the algorithm has some confidence about their worth relative to tradition. This can be a very good thing, although it needs to be done with caution.

## Running Better Simulations

In addition to initialising your algorithm using prior information you have before de‐ ploying a Bandit algorithm, you can often run much better simulations if you use historical information to build appropriate simulations. In this book we’ve used a toy Monte Carlo simulation with click-through rates that varied from `0.1` to `0.9`. Real world click- through rates are typically much lower than this. Because low success rates may mean that your algorithm must run for a very long time before it is able to reach any strong conclusions, you should conduct simulations that are informed by real data about your business if you have access to it.

## Moving Worlds

In the real world, the value of different arms in a bandit problem can easily change over time. As we said in the introduction, an orange and black site design might be perfect during Halloween, but terrible during Christmas. Because the true value of an arm might actually shift over time, you want your estimates to be able to do this as well.

_Arms with changing values can be a very serious problem if you’re not careful when you deploy a bandit algorithm_. The algorithms we’ve presented will not handle most sorts of change in the underlying values of arms well. The problem has to do with the way that we estimate the value of an arm. We typically updated our estimates using the following snippet of code:

```
new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
self.values[chosen_arm] = new_value
```

The problem with this update rule is that `1 / float(n)` goes to `0` as n gets large. When you’re dealing with millions or billions of plays, this means that recent rewards will have almost zero effect on your estimates of the value of different arms. If those values shifted only a small amount, the algorithm will take a huge number of plays to update its estimated values.

There is a simple trick for working around this that can be used if you’re careful: instead of estimating the values of the arms using strict averages, you can overweight recent events by using a slightly different update rule based on a different snippet of code:

```
new_value = (1 - alpha) * value + (alpha) * reward
self.values[chosen_arm] = new_value
```

In the traditional rule, alpha changed from trial to trial. In this alternative, alpha is a fixed value between `0.0` and `1.0`. This alternative updating rule will allow your estimates to shift much more with recent experiences. When the world can change radically, that flexibility is very important.

Unfortunately, the price you pay for that flexibility is the introduction of a new parameter that you’ll have to tune to your specific business. We encourage you to experiment with this modified updating rule using simulations to develop an intuition for how it behaves in environments like yours. If used appropriately in a changing world, setting alpha to a constant value can make a big difference relative to allowing alpha to go to `0` too quickly. But, if used carelessly, this same change will make your algorithm behave er‐ ratically. If you set `alpha = 1.0`, you can expect to unleash a nightmare for yourself.

## Correlated Bandits

In many situations, you want to solve a Multiarmed Bandit Problem with a large number of arms. This will be hopeless unless there is some way you can generalize your experiences with some arms to other arms. When you can make generalisations safely, we say that the arms are correlated. To be extremely precise, what matters is that the expected rewards of different arms are correlated.

To illustrate this idea, let’s go back to our earlier idea about experimenting with different color logos. It’s reasonable to assume that similar colors are likely to elicit similar reactions. So you might try to propagate information about rewards from one color to other colors based on their degree of similarity.

If you’re working with thousands of colors, simple algorithms like UCB1 may not be appropriate because they can’t exploit the correlations across colors. You’ll need to find ways to relate arms and update your estimates based on this information. We encourage you to look into classical smoothing techniques in statistics to get a sense for how you might deal with correlated arms.

## Contextual Bandits

In addition to correlations between arms in a bandit task, it’s often the case that we have background information about the context in which we’re trying out different options. For example, we may find that certain fonts are more appealing to male users than to female users. We refer to this side information as context. There are a variety of algo‐ rithms like `LinUCB` and `GLMUCB` for working with contextual information: you can read about them in two academic papers called “A Contextual-Bandit Approach to Per‐ sonalized News Article Recommendation” and “Parametric Bandits: The Generalized Linear Case”.

Both of these algorithms are more complicated than the algorithms we’ve covered in this book, but the spirit of these models is easy to describe: you want to develop a predictive model of the value of arms that depends upon context. You can use any of the techniques available in conventional machine learning for doing this. If those techniques allow you to update your model using online learning, you can build a contextual bandit algorithm out of them.

`LinUCB` does this by updating a linear regression model for the arms’ values after each play. `GLMUCB` does this by updating a General Linear Model for the arms’ values after each play. Many other algorithms exist and you could create your own with some research into online versions of your favorite machine learning algorithm.

## Implementing Bandit Algorithms at Scale

Many of the topics we’ve discussed make bandit algorithms more complex in order to cope with the complexity of the real world. But that complexity may make deploying a bandit algorithm prohibitively difficult at scale. Why is that?

Even in the simplest real-world settings, the bandit algorithms we’ve described in this book may not work as well as they do in simulations because you often may not know what happened on your N-th play in the real world until a while after you’ve been forced to serve a new page for (and therefore select a new arm for) many other users. This destroys the clean sequential structure we’ve assumed throughout the book. If you’re a website that serves hundreds of thousands of hits in a second, this can be a very substantial break from the scenarios we’ve been envisoning.

This is only one example of how the algorithms we’ve described are non-trivial when you want to get them to scale up, but we’ll focus on it for the sake of brevity. Our proposed solution seems to be the solution chosen by Google for Google Analytics based on information in their help documents, although we don’t know the details of how their system is configured.

In short, our approach to dealing with imperfect sequential assignments is to embrace this failure and develop a system that is easier to scale up. We propose doing this in two parts:
- Blocked assignments
    - Assign incoming users to new arms in advance and draw this information from a fast cache when users actually arrive. Store their responses for batch processing later in another fast cache.
- Blocked updates
    - Update your estimates of arm values in batches on a regular interval and regenerate your blocked assignments. Because you work in batches, it will be easier to perform the kind of complex calculations you’ll need to deal with correlated arms or contextual information.
    
Changes like this can go a long way in making bandit algorithms scale up for large websites. But, once you start to make changes to bandit algorithms to deal with these sorts of scale problems, you’ll find that the theoretical literature on bandits often be‐ comes less informative about what you can expect will happen. There are a few papers that have recently come out: if you’re interested, this problem is referred to as the problem of _delayed feedback_ in the academic literature.

Thankfully, even though the academic literature is a little sparser on the topic of delayed feedback, you can still run Monte Carlo simulations to test your approach before deploying a bandit system that has to cope with delayed feedback. Of course, you’ll have to make simulations that are more complex than those we’ve described already, but those more complex simulations are still possible to design. And they may convince you that your proposed algorithms works even though you’re working in uncharted waters beyond what theoreticians have studied. That’s the reason we’ve focused on using simulations through the book. We want you to feel comfortable exploring this topic for yourself, even when doing so will take you into areas that science hasn’t fully reached yet.

While you’re exploring, you’ll come up with lots of other interesting questions about scaling up bandit algorithms like:
- What sort of database should you store information in? Is something like MySQL usable or do you need to work with something like Memcached? If you need to pull out assignments to arms quickly, it’s probably wise to move this information into the lowest latency data storage tool you have available to you.
- Where in your production code should you be running the equivalent of our `select_arm` and `update` functions? In the blocked assignments model we described earlier, this happens far removed from the tools that directly generate served pages. But in the obvious strategy for deploying bandit algorithms, this happens in the page generation mechanism itself.